# Test MCP Launching

In [1]:
from open_instruct.grpo_fast import launch_mcp_subprocess
run_mcp_command="cd ../rl-rag-mcp && python -m mcp_agents.mcp_backend.main --transport http --port 8000 --path /mcp"
launch_mcp_subprocess(run_mcp_command, "output")

[2025-09-10 20:05:41,769] [WARNING] [real_accelerator.py:174:get_accelerator] Setting accelerator to CPU. If you have GPU or other accelerator, we were unable to detect it.
[2025-09-10 20:05:41,774] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cpu (auto detect)


/home/rulin/miniconda3/envs/open-instruct/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-10 20:05:46,290	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 09-10 20:05:51 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 09-10 20:05:51 [__init__.py:243] No platform detected, vLLM is running on UnspecifiedPlatform
WARNING 09-10 20:05:51 [_custom_ops.py:21] Failed to import from vllm._C with ImportError('libcuda.so.1: cannot open shared object file: No such file or directory')
🚀 Launching MCP server subprocess: cd ../rl-rag-mcp && python -m mcp_agents.mcp_backend.main --transport http --port 8000 --path /mcp
✅ Found fastmcp at: /home/rulin/miniconda3/envs/open-instruct/bin/fastmcp
📁 Current working directory: /checkpoint/comem/rulin/open-instruct
✅ MCP server started successfully (PID: 228837)
📋 MCP server logs: output/mcp_logs/mcp_server_stdout.log, output/mcp_logs/mcp_server_stderr.log


<Popen: returncode: None args: ['cd ../rl-rag-mcp && python -m mcp_agents.mc...>

# Debug FGRPO Toy Case

In [2]:
import json
rl_log_file = "/checkpoint/comem/rulin/open-instruct/output/debug_rl_rag_fgrpo_toy_2wiki_5q_same_group_massive_serve__1__1757530995/traces_debug_rl_rag_fgrpo_toy_2wiki_5q_same_group_massive_serve__1__1757530995.jsonl"
data = [json.loads(line) for line in open(rl_log_file)]

In [3]:
data[0].keys()

dict_keys(['advantages', 'finish_reasons', 'responses', 'queries', 'ground_truths', 'datasets', 'training_step', 'objective/reward_log_values/question_0_accuracy', 'objective/reward_log_values/question_1_accuracy', 'objective/reward_log_values/question_2_accuracy', 'objective/reward_log_values/question_3_accuracy', 'objective/reward_log_values/question_4_accuracy', 'objective/reward_log_values/num_questions', 'objective/reward_log_values/averaged_accuracy', 'objective/reward_log_values/num_valid_spans'])

In [4]:
print("Number of responses:", len(data[0]["responses"]))
print("Example ground truth:", data[0]["ground_truths"][0])

Number of responses: 128
Example ground truth: ['no; Brooklyn; no; no; no']


In [5]:
# Decode the response and query
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("ai2-adapt-dev/tulu_3_long_finetune_qwen_7b_reg")

/home/rulin/miniconda3/envs/open-instruct/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
res, query = data[0]["responses"][0], data[0]["queries"][0]
response = tokenizer.decode(res, skip_special_tokens=True)
question = tokenizer.decode(query, skip_special_tokens=True)
ground_truth = json.dumps(data[0]["ground_truths"][0])

In [7]:
ground_truth

'["no; Brooklyn; no; no; no"]'

In [8]:
from typing import List
def extract_ground_truth_per_question(ground_truth: str) -> List[str]:
    """
    Extract ground truth answers for each individual question.
    Expects ground truth in format: JSON arrays separated by semicolons
    e.g., '["ans1", "ans2"]; ["ans3", "ans4"]; ["ans5"]'
    Returns: List of JSON strings, one per question
    """
    ground_truth = json.loads(ground_truth)
    if len(ground_truth) != 1:
        return [json.dumps(item) if isinstance(item, list) else str(item) for item in ground_truth]
    else:
        ground_truth = ground_truth[0]
        return ground_truth.split(";")  # TODO: the use of this is terrible


In [9]:
from typing import Tuple, List
import re
def split_response_and_get_spans(response: str, num_questions: int) -> Tuple[List[str], List[List[Tuple[int, int]]]]:
    """
    Split a multi-question response into individual question responses and get their spans.
    Splits by </answer{i}> tags where i is the question number (1-indexed).
    
    Args:
        response: The full response containing multiple answers
        num_questions: Number of questions expected
        
    Returns:
        Tuple of (sub_responses, spans) where:
        - sub_responses: List of response text for each question
        - spans: List of span tuples [(start_char, end_char)] for each question
    """
    if num_questions <= 1:
        return [response], [[(0, len(response))]], 1
    
    sub_responses = []
    spans = []
    num_valid_spans = 0
    
    # Find split points using </answer{i}> tags
    start_char = 0
    for i in range(1, num_questions + 1):
        end_tag = f"</answer{i}>"
        match = re.search(re.escape(end_tag), response[start_char:])
        if match:
            end_char = start_char + match.end()
            sub_responses.append(response[start_char:end_char])
            spans.append([(start_char, end_char)])
            num_valid_spans += 1
            start_char = end_char
        elif i == num_questions:
            sub_responses.append(response[start_char:])
            spans.append([(start_char, len(response))])
            break
        else:
            # # Otherwise, return invalid spans that will trigger the fallback mechanism to use the entire response
            # sub_responses.append(response)
            # spans.append([(-1, -1)])
            # Or, penalize the entire response and return empty sub_response
            sub_responses.append("")
            spans.append([(-1, -1)])
        
    return sub_responses, spans, num_valid_spans

In [10]:
ground_truth_per_question = extract_ground_truth_per_question(ground_truth)
num_questions = len(ground_truth_per_question)
sub_responses, spans, num_valid_spans = split_response_and_get_spans(response, num_questions)

In [11]:
num_valid_spans

3

In [12]:
for sub_response, span in zip(sub_responses, spans):
    print(sub_response)
    print(span)
    print("="*100)


<answer1>\boxed{No}</answer1>
[(0, 29)]

<search>Do Sucker Punch (2011 film) and Plaga Zombie share the same director nationality?</search>
<snippets id=4fbd077c>
Plaga Zombie (film series) studio based in New Bedford, Massachusetts, the company is best known for its award-winning 2007 horror film "The Terror Factor". Writer-director Garry Mederios discovered the Plaga Zombie series after watching the Fangoria Film DVD. Mederios enjoyed the series so much that he contacted the filmmakers via e-mail and became friends. He jokingly suggested making an American-based Plaga Zombie film during one of their exchanges but FARSA Productions was very receptive to the idea and plans were eventually made to produce a feature-length film. Set in the same universe as the original trilogy, Plaga Zombie: American Invasion is to feature three

Nathan Schiff The poetic, whirling, free style of its imagery is remarkably close in spirit to James Joyce". Schiff later returned to filmmaking with the horror

In [13]:
len(response)

7484

In [14]:
from typing import Union
from open_instruct.search_rewards.toy_case_multi_dataset_reward import normalize_answer, extract_boxed_answer_from_response
def verify_one_question(response: str, target: str, use_exact_match: bool = False) -> float:
    """
    Verify a single question response against ground truth.
    
    Args:
        response: The response text for one question
        ground_truth: Ground truth data for this question
        use_exact_match: If True, use exact match; if False, use contains match
        
    Returns:
        Score between 0.0 and 1.0
    """
    if not response or not target:
        return 0.0
    
    # check if the response is a list of answers
    parsed_labels: Union[List, str]
    try:
        parsed = json.loads(target)
        parsed_labels = parsed if isinstance(parsed, list) else [parsed]
    except (json.JSONDecodeError, TypeError):
        # Fallback: treat label as raw string or list-of-strings
        if isinstance(target, list):
            parsed_labels = target
        else:
            parsed_labels = [str(target).strip()]
    
    for label in parsed_labels:
        # Normalize both strings for comparison
        response_normalized = normalize_answer(extract_boxed_answer_from_response(response.strip()))
        target_normalized = normalize_answer(str(label).strip())
        print("extracted answer:", response_normalized)
        print("ground truth:", target_normalized)
        
        if use_exact_match:
            # Exact match after normalization
            if response_normalized == target_normalized:
                return 1.0
        else:
            # Contains match - check if ground truth is contained in response
            if target_normalized.lower() in response_normalized.lower():
                return 1.0
    return 0.0

In [15]:
from typing import Dict, Any, Optional
from open_instruct.search_rewards.toy_case_multi_dataset_reward import FinegrainedScore

def compute_multi_question_reward(
    response: str, 
    ground_truth: str, 
    query: Optional[str] = None,
    use_exact_match: bool = False,
    reward_type: str = "finegrained",
    ) -> Dict[str, Any]:
    """
    Compute finegrained multi-question reward with simple verifiable scoring and spans.
    
    Args:
        response: The full response containing multiple answers
        ground_truth: Dictionary containing ground truth data
        use_exact_match: If True, use exact match; if False, use contains match
        
    Returns:
        Dict with:
            - finegrained_scores: List of (score, (start_char, end_char), reward_group_id, response_idx) tuples
            - log_values: Dict of metrics for logging
    """
    # Get verifiable reward scores for each question
    ground_truth_per_question = extract_ground_truth_per_question(ground_truth)
    print(f"🎀 ground_truth_per_question: {ground_truth_per_question}")
    num_questions = len(ground_truth_per_question)
    
    # Split the response into individual question components for span generation
    sub_responses, spans, num_valid_spans = split_response_and_get_spans(response, num_questions)
    
    # Compute finegrained scores for each question
    finegrained_scores = []
    for i, (sub_response, gt, span) in enumerate(zip(sub_responses, ground_truth_per_question, spans)):
        sub_score = verify_one_question(sub_response, gt, use_exact_match)

        group_id = 0  # rulin: assume the same group for all questions
        finegrained_scores.append(
            FinegrainedScore(
                score=sub_score,
                effective_spans=span,
                reward_group_id=group_id,  
                reward_group_name=f"question_{group_id}",
            )
        )
    
    averaged_score = sum(item.score for item in finegrained_scores) / len(finegrained_scores)
    
    # Create log values for tracking
    log_values = {
        **{f"question_{i}_accuracy": item.score for i, item in enumerate(finegrained_scores)},
        "num_questions": num_questions,
        "averaged_accuracy": averaged_score,
        "num_valid_spans": num_valid_spans,
    }
    
    if reward_type == "finegrained":
        print(f"🎀 finegrained_scores: {log_values}")
        return {
            "finegrained_scores": finegrained_scores,
            "log_values": log_values,
        }
    elif reward_type == "averaged":
        print(f"🎀 averaged_score: {averaged_score}")
        return {
            "score": averaged_score,
            "log_values": log_values,
        }
    else:
        raise ValueError(f"Invalid reward type: {reward_type}")

In [16]:
reward_per_question = compute_multi_question_reward(
    response,
    ground_truth,
    query,
    use_exact_match=False,
    reward_type="finegrained",
)


🎀 ground_truth_per_question: ['no', ' Brooklyn', ' no', ' no', ' no']
extracted answer: no
ground truth: no
extracted answer: palermo sicily
ground truth: brooklyn
extracted answer: no
ground truth: no
extracted answer: 
ground truth: no
🎀 finegrained_scores: {'question_0_accuracy': 1.0, 'question_1_accuracy': 0.0, 'question_2_accuracy': 1.0, 'question_3_accuracy': 0.0, 'question_4_accuracy': 0.0, 'num_questions': 5, 'averaged_accuracy': 0.4, 'num_valid_spans': 3}


In [17]:
advantage = data[0]["advantages"][0]
len(advantage)

1742

In [18]:
len(res)

1742

In [19]:
from open_instruct.search_rewards.utils.span_convertion import (
    message_span_aware_tokenization,
    convert_string_spans_to_token_spans
)
import logging


def convert_string_span_to_token_span(effective_spans, decoded_resp, token_resp, tokenizer):
    """
    Convert character spans to token spans and create a mask for training.
    Uses the span conversion logic from open_instruct/search_rewards/span_convertion.py
    
    Args:
        effective_spans: List of tuples (start_char, end_char) in decoded response
        decoded_resp: Decoded string response
        token_resp: Tokenized response (list of token IDs)
        tokenizer: Tokenizer used to decode the responses
    
    Returns:
        Tuple of (mask, span_mapping_stats) where:
        - mask: List of integers where 1 means the token should be trained on, 0 means masked
        - span_mapping_stats: Dict with mapping quality statistics
    """

    
    logger = logging.getLogger(__name__)
    
    # Initialize statistics tracking
    span_stats = {
        "total_spans": len(effective_spans) if effective_spans else 0,
        "valid_start_mappings": 0,
        "valid_end_mappings": 0,
        "fallback_start_mappings": 0,
        "fallback_end_mappings": 0,
    }
    
    if not effective_spans or len(token_resp) == 0:
        # If no effective spans or no tokens, mask everything except last EOS
        mask = [0] * len(token_resp)
        if len(token_resp) > 0 and token_resp[-1] == tokenizer.eos_token_id:
            mask[-1] = 1
        return mask, span_stats
    
    # Use the span conversion utilities directly without creating dummy messages
    # We'll tokenize the decoded response directly and get offset mappings
    tokenized_resp = tokenizer(
        decoded_resp,
        return_tensors="pt",
        return_offsets_mapping=True,
        add_special_tokens=False,
    )
    
    # Check for empty tokenization
    if len(tokenized_resp["input_ids"][0]) == 0:
        logger.warning(f"Empty tokenization for decoded_resp: '{decoded_resp[:100]}...'")
        # Return all-masked except EOS
        mask = [0] * len(token_resp)
        if len(token_resp) > 0 and token_resp[-1] == tokenizer.eos_token_id:
            mask[-1] = 1
        return mask, span_stats
    
    # Create a simple tokenized_input structure that the conversion function expects
    tokenized_input = {
        "offset_mapping": tokenized_resp["offset_mapping"][0],  # Remove batch dimension
        "message_token_spans": {0: [0, len(tokenized_resp["input_ids"][0])]},  # Single message spanning all tokens
    }
    
    # Convert string spans to token spans
    token_spans = convert_string_spans_to_token_spans(
        tokenized_input,
        effective_spans,
        target_message_span_index=0
    )
    
    # Initialize mask - everything masked by default
    mask = [0] * len(token_resp)
    
    # Update statistics and create mask
    for span_idx, (token_start, token_end) in enumerate(token_spans):
        if token_start is not None and token_end is not None:
            span_stats["valid_start_mappings"] += 1
            span_stats["valid_end_mappings"] += 1
            
            # Clamp to valid token range for our response
            token_start = max(0, min(token_start, len(token_resp)))
            token_end = max(token_start, min(token_end, len(token_resp)))
            
            # Unmask tokens in this span
            for token_idx in range(token_start, token_end):
                if token_idx < len(mask):
                    mask[token_idx] = 1
        else:
            span_stats["fallback_start_mappings"] += 1
            span_stats["fallback_end_mappings"] += 1
            logger.warning(f"Span {span_idx} could not be converted to token span. No reward will be applied.")
    
    # Always keep the last EOS token unmasked if it exists
    if len(token_resp) > 0 and token_resp[-1] == tokenizer.eos_token_id:
        mask[-1] = 1
    
    return mask, span_stats

In [24]:
print(set(advantage))
print(spans)
print("*"*100)
mask_list, span_stats_list, decoded_list = [], [], []
for i in range(len(spans)):
    print(spans[i])
    if spans[i][0][0] != -1:
        mask, span_stats = convert_string_span_to_token_span(effective_spans=spans[i], decoded_resp=response, token_resp=res, tokenizer=tokenizer)
        decoded_res = tokenizer.decode(res[mask.index(1):], skip_special_tokens=True)
        mask_list.append(mask)
        span_stats_list.append(span_stats)
        decoded_list.append(decoded_res)
        print(len(mask))
        print(len(res))
        print(span_stats)
        print(decoded_res)
        
    print("="*100)

{0.9276126027107239, 0.0, -0.2757767140865326, -1.0780363082885742}
[[(0, 29)], [(29, 2602)], [(2602, 5401)], [(-1, -1)], [(5401, 7484)]]
****************************************************************************************************
[(0, 29)]
1742
1742
{'total_spans': 1, 'valid_start_mappings': 1, 'valid_end_mappings': 1, 'fallback_start_mappings': 0, 'fallback_end_mappings': 0}
<answer1>\boxed{No}</answer1>
<search>Do Sucker Punch (2011 film) and Plaga Zombie share the same director nationality?</search>
<snippets id=4fbd077c>
Plaga Zombie (film series) studio based in New Bedford, Massachusetts, the company is best known for its award-winning 2007 horror film "The Terror Factor". Writer-director Garry Mederios discovered the Plaga Zombie series after watching the Fangoria Film DVD. Mederios enjoyed the series so much that he contacted the filmmakers via e-mail and became friends. He jokingly suggested making an American-based Plaga Zombie film during one of their exchanges but 

In [34]:
spans[0], mask_list[0], [res[i] for i in range(len(res)) if mask_list[0][i] == 1]

([(0, 29)],
 [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0

In [39]:
j = 0
effective_tokens = [res[i] for i in range(len(res)) if mask_list[j][i] == 1]
tokenizer.decode(effective_tokens, skip_special_tokens=True)

'<answer1>\\boxed{No}</answer1'

In [40]:
j = 1
effective_tokens = [res[i] for i in range(len(res)) if mask_list[j][i] == 1]
tokenizer.decode(effective_tokens, skip_special_tokens=True)

'<search>Do Sucker Punch (2011 film) and Plaga Zombie share the same director nationality?</search>\n<snippets id=4fbd077c>\nPlaga Zombie (film series) studio based in New Bedford, Massachusetts, the company is best known for its award-winning 2007 horror film "The Terror Factor". Writer-director Garry Mederios discovered the Plaga Zombie series after watching the Fangoria Film DVD. Mederios enjoyed the series so much that he contacted the filmmakers via e-mail and became friends. He jokingly suggested making an American-based Plaga Zombie film during one of their exchanges but FARSA Productions was very receptive to the idea and plans were eventually made to produce a feature-length film. Set in the same universe as the original trilogy, Plaga Zombie: American Invasion is to feature three\n\nNathan Schiff The poetic, whirling, free style of its imagery is remarkably close in spirit to James Joyce". Schiff later returned to filmmaking with the horror short "Abracadaver!" (2008), made f

In [41]:
j = 2
effective_tokens = [res[i] for i in range(len(res)) if mask_list[j][i] == 1]
tokenizer.decode(effective_tokens, skip_special_tokens=True)

'<search>Where was the director of Steel (1979 Film) born?</search>\n<snippets id=e2290384>\nDon Levy position at the Carpenter Center for the Visual Arts at Harvard University, where he stayed for two years. He then moved to Los Angeles to work at the California Institute of the Arts, where he taught and conducted research in film, video and multimedia until his death in 1987 by committing suicide. Don Levy Don Levy (1932 – January 1987) was an artist and filmmaker. Levy was born in Bathurst, New South Wales, Australia. After studying theoretical chemistry at the University of Sydney, he was awarded a Research Scholarship to the University of Cambridge. There he obtained a PhD in\n\nRoger Sherman (filmmaker) in 1969. He attended Union College (1969-1971), majoring in political science and experimental education, played freshman soccer, and spent a semester in Bogotá, Colombia where he studied Spanish. He left Union and went to Denmark for his junior year abroad at The University of Co

In [42]:
advantage

[0.9276126027107239,
 0.9276126027107239,
 0.9276126027107239,
 0.9276126027107239,
 0.9276126027107239,
 0.9276126027107239,
 0.9276126027107239,
 0.9276126027107239,
 0.9276126027107239,
 0.9276126027107239,
 0.0,
 -1.0780363082885742,
 -1.0780363082885742,
 -1.0780363082885742,
 -1.0780363082885742,
 -1.0780363082885742,
 -1.0780363082885742,
 -1.0780363082885742,
 -1.0780363082885742,
 -1.0780363082885742,
 -1.0780363082885742,
 -1.0780363082885742,
 -1.0780363082885742,
 -1.0780363082885742,
 -1.0780363082885742,
 -1.0780363082885742,
 -1.0780363082885742,
 -1.0780363082885742,
 -1.0780363082885742,
 -1.0780363082885742,
 -1.0780363082885742,
 -1.0780363082885742,
 -1.0780363082885742,
 -1.0780363082885742,
 -1.0780363082885742,
 -1.0780363082885742,
 -1.0780363082885742,
 -1.0780363082885742,
 -1.0780363082885742,
 -1.0780363082885742,
 -1.0780363082885742,
 -1.0780363082885742,
 -1.0780363082885742,
 -1.0780363082885742,
 -1.0780363082885742,
 -1.0780363082885742,
 -1.0780363082

In [43]:
mask_list[0]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [44]:
from datasets import load_dataset
dataset = load_dataset("rulins/multi_question_synthetic_single_source_2wiki_5q", split="train")

Generating validation split: 100%|██████████| 100/100 [00:00<00:00, 15325.58 examples/s]


In [45]:
gt = dataset[0]["ground_truth"]

In [47]:
import json
gt_list = json.loads(gt)

In [49]:
gt_list[:3]

['California Institute of the Arts', 'no', 'no']